# Forest Fire Detection - Transfer Learning Model

This notebook implements transfer learning by first training on the Github dataset and then fine-tuning on the Mendley dataset.

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
# Set paths for Linux environment
GITHUB_PATH = '../Data/ImageDataset/Github'
MENDLEY_PATH = '../Data/ImageDataset/Mendley/Forest Fire Dataset'

In [ ]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=0.2,
    fill_mode='nearest'
)

valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Build base model using ResNet50
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Create the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

## Phase 1: Training on Github Dataset

In [ ]:
# Prepare Github dataset
github_train = train_datagen.flow_from_directory(
    os.path.join(GITHUB_PATH, 'train'),
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

github_valid = valid_datagen.flow_from_directory(
    os.path.join(GITHUB_PATH, 'valid'),
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5,
    min_lr=0.00001
)

# Train on Github dataset
github_history = model.fit(
    github_train,
    epochs=30,
    validation_data=github_valid,
    callbacks=[early_stopping, reduce_lr]
)

## Phase 2: Fine-tuning on Mendley Dataset

In [ ]:
# Unfreeze some layers of the base model
for layer in model.layers[0].layers[-30:]:  # Unfreeze last 30 layers of ResNet50
    layer.trainable = True

# Recompile with a lower learning rate
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Prepare Mendley dataset
mendley_train = train_datagen.flow_from_directory(
    os.path.join(MENDLEY_PATH, 'train'),
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

mendley_valid = valid_datagen.flow_from_directory(
    os.path.join(MENDLEY_PATH, 'valid'),
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

mendley_test = test_datagen.flow_from_directory(
    os.path.join(MENDLEY_PATH, 'test'),
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Fine-tune on Mendley dataset
mendley_history = model.fit(
    mendley_train,
    epochs=20,
    validation_data=mendley_valid,
    callbacks=[early_stopping, reduce_lr]
)

In [ ]:
# Plot combined training history
def plot_combined_history(github_history, mendley_history):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Combine histories
    total_epochs = len(github_history.history['accuracy']) + len(mendley_history.history['accuracy'])
    epochs_range = range(1, total_epochs + 1)
    
    # Accuracy plot
    ax1.plot(range(1, len(github_history.history['accuracy']) + 1),
             github_history.history['accuracy'],
             label='Github Training')
    ax1.plot(range(1, len(github_history.history['val_accuracy']) + 1),
             github_history.history['val_accuracy'],
             label='Github Validation')
    ax1.plot(range(len(github_history.history['accuracy']) + 1,
                   total_epochs + 1),
             mendley_history.history['accuracy'],
             label='Mendley Training')
    ax1.plot(range(len(github_history.history['accuracy']) + 1,
                   total_epochs + 1),
             mendley_history.history['val_accuracy'],
             label='Mendley Validation')
    ax1.set_title('Model Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    
    # Loss plot
    ax2.plot(range(1, len(github_history.history['loss']) + 1),
             github_history.history['loss'],
             label='Github Training')
    ax2.plot(range(1, len(github_history.history['val_loss']) + 1),
             github_history.history['val_loss'],
             label='Github Validation')
    ax2.plot(range(len(github_history.history['loss']) + 1,
                   total_epochs + 1),
             mendley_history.history['loss'],
             label='Mendley Training')
    ax2.plot(range(len(github_history.history['loss']) + 1,
                   total_epochs + 1),
             mendley_history.history['val_loss'],
             label='Mendley Validation')
    ax2.set_title('Model Loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()

plot_combined_history(github_history, mendley_history)

In [ ]:
# Evaluate final model
test_loss, test_accuracy = model.evaluate(mendley_test)
print(f'Final test accuracy: {test_accuracy:.4f}')

# Save the final model
model.save('../models/transfer_learning_model.h5')